# EOFs package NAO example (xarray)

Compute and plot the leading EOF of geopotential height on the 500 hPa
pressure surface over the European/Atlantic sector during winter time.

This example uses the metadata-retaining xarray interface.

Additional requirements for this example:

    * xarray (http://xarray.pydata.org)
    * matplotlib (http://matplotlib.org/)
    * cartopy (http://scitools.org.uk/cartopy/)

This notebook is based on code in

    https://ajdawson.github.io/eofs/latest/examples/nao_xarray.html

Change history:

MGH 2019-04-15
    - Notebook adapted from https://git.niwa.local/hadfield/python-mgh/blob/master/examples/mgh_example_eofs_nao_xarray.py

In [2]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

from eofs.xarray import Eof
from eofs.examples import example_data_path

Read geopotential height data using the xarray module.

The file contains December-February averages of geopotential height at
500 hPa for the European/Atlantic domain (80W-40E, 20-90N).

In [3]:
filename = example_data_path('hgt_djf.nc')
z_djf = xr.open_dataset(filename)['z']

Compute anomalies by removing the time-mean.

In [4]:
z_djf = z_djf - z_djf.mean(dim='time')

Create an EOF solver to do the EOF analysis.

Square-root of cosine of latitude weights are applied before the
computation of EOFs.

In [5]:
coslat = np.cos(np.deg2rad(z_djf.coords['latitude'].values)).clip(0., 1.)
wgts = np.sqrt(coslat)[..., np.newaxis]
solver = Eof(z_djf, weights=wgts)

Retrieve the leading EOF

The EOF is expressed as the covariance between the leading PC
time series and the input SLP anomalies at each grid point.

In [8]:
eof1 = solver.eofsAsCovariance(neofs=1)

Plot the leading EOF

In [7]:
plt.figure()
clevs = np.linspace(-75, 75, 11)
proj = ccrs.Orthographic(central_longitude=-20, central_latitude=60)
ax = plt.axes(projection=proj)
ax.coastlines()
ax.set_global()
eof1[0, 0].plot.contourf(ax=ax, levels=clevs, cmap=plt.cm.RdBu_r,
                         transform=ccrs.PlateCarree(), add_colorbar=False)
ax.set_title('EOF1 expressed as covariance', fontsize=16)

plt.show()